In [1]:
# Зависимости
import gym
import numpy
from collections import deque
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.optimizers import Adam
from random import sample

Using TensorFlow backend.


In [2]:
# Создание нейронной сети, которая для заданного состояния S вычисляет значения Q(s,a) для всех возможных a.
def create_model(state_size, num_actions) :
    learning_rate = 0.01

    # Структура сети
    input_layer_size = state_size
    hidden_layers_size = [10, 10, 10]
    output_layer_size = num_actions
    
    # Построение сети
    model = Sequential()
    model.add(Dense(hidden_layers_size[0], input_dim=input_layer_size, activation="relu"))
    for layer in range(1, len(hidden_layers_size)) :
        model.add(Dense(hidden_layers_size[layer], activation="relu"))
    model.add(Dense(output_layer_size))
    
    model.compile(loss="mean_squared_error", optimizer=Adam(lr=learning_rate))
    
    return model

In [3]:
# Обучение сети с использованием ранее заполненной памяти
def fit_on_memory(model, memory, state_size):
    # Размер батча для извлечения из памяти
    batch_size = 16
    
    # Коэффициент уменьшения (дисконта)
    gamma = 0.95

    # Случайное получение батча, чтобы обучаться на нём
    batch = sample(memory, min(len(memory), batch_size))

    # Извлечение значений из батча
    actions, rewards, results = [], [], []
    states = numpy.zeros((batch_size, state_size))
    next_states = numpy.zeros((batch_size, state_size))

    for i in range(batch_size) :
        states[i] = batch[i][0]
        actions.append(batch[i][1])
        rewards.append(batch[i][2])
        next_states[i] = batch[i][3]
        results.append(batch[i][4])

    # Вычисление значений Q для текущего и следующего состояний
    q_values = model.predict(states)
    q_next_values = model.predict(next_states)

    # Коррекция значений Q
    for t in range(batch_size) :
        # Если мы на последнем шаге, то значение Q равно полученному подкреплению
        if results[t] == True :
            q_values[t][actions[t]] = rewards[t]
        # Иначе мы учитываем дисконтированный максимум из следующих значений Q
        else :
            q_values[t][actions[t]] = rewards[t] + gamma * (numpy.amax(q_next_values[t]))

    # Обучение сети
    model.fit(states, q_values, batch_size=batch_size, verbose=0)

In [4]:
# Выполнение действий в окружающей среде, заполнение памяти и обучение на ней
def try_and_learn(env, model, model_name, num_runs, save_condition, logs = True) :
    # Параметры памяти
    maximum_memory_size = 500
    memory = deque(maxlen = maximum_memory_size)
    minimum_memory_size_for_learning = 50
    
    # Параметры выбора действия
    epsilon = 1.0
    epsilon_min = 0.01
    epsilon_decay = 0.9995
    
    # Значение по умолчанию для отрицательного подкрепления, если итерация закончилась преждевременно
    negative_reward = -100

    state_size = env.observation_space.shape[0]

    # В каждой итерации мы пытаемся обучить сеть, чтобы достичь заданный макисмум эффективности
    for run in range(num_runs) :
        env_state = env.reset()
        state = numpy.reshape(env_state, [1, state_size])
        done = False
        steps_survived = 0
        while not done :
            # Сначала действие выбирается случайным образом,
            # но если эпсилон-жадное правило позволяет, то действие выбирается в соответствии с прогнозом сети
            action = env.action_space.sample()
            random_pick = numpy.random.random()
            if random_pick > epsilon :
                action = numpy.argmax(model.predict(state))

            # Если память достаточно велика, мы начинаем уменьшать эпсилон
            if len(memory) >= minimum_memory_size_for_learning and epsilon > epsilon_min :
                epsilon = max(epsilon * epsilon_decay, epsilon_min)
            
            # Взаимодействие с окружающей средой
            env_next_state, reward, done, info = env.step(action)
            next_state = numpy.reshape(env_next_state, [1, state_size])
            
            # Если итерация завершена, но сеть прожила меньше заданного максимума, то подкрепление отрицательное
            if done and steps_survived != env._max_episode_steps - 1 :
                reward = negative_reward
            
            # Добавление записи в память
            memory.append((state, action, reward, next_state, done))
            
            # Переход в следующее состояние
            state = next_state
            steps_survived += 1
            
            # Если итерация завершена, то выводятся некоторые логи
            if done :                   
                if logs : print("run: {}/{}, score: {}/{}, eps: {:.6}, len(memory): {}".format(run, num_runs, steps_survived, save_condition, epsilon, len(memory)))
                
                # Если сеть прожила больше заданного значения, то она сохраняется в файл и обучение заканчивается
                if steps_survived >= save_condition :
                    if logs : print("Saving the model as {}".format(model_name))
                    model.save(model_name)
                    return run
            
            # Если память достаточно велика, начиаем обучение
            if len(memory) >= minimum_memory_size_for_learning :
                fit_on_memory(model, memory, env.observation_space.shape[0])
    # Если код дошёл до этой строки, значит сеть не смогла обучиться
    if logs : print("The model is not good enough")
    return -1

In [5]:
# Тестирование поведения обученной сети
def test(env, model_name, num_runs, render = True, logs = True):
    # Загрузка сети
    model = load_model(model_name)
    
    state_size = env.observation_space.shape[0]
    
    sum_result = 0
    complete_success = 0
    
    for run in range(num_runs) :
        env_state = env.reset()
        state = numpy.reshape(env_state, [1, state_size])
        done = False
        steps_survived = 0
        while not done :
            if render :
                env.render()
            
            action = numpy.argmax(model.predict(state))
            
            env_next_state, reward, done, info = env.step(action)
            state = numpy.reshape(env_next_state, [1, state_size])
            
            steps_survived += 1
            
            if done :
                if logs : print("run: {}/{}, score: {}".format(run, num_runs, steps_survived))
                sum_result += steps_survived
                if steps_survived == env._max_episode_steps :
                    complete_success += 1
                break
    return sum_result / num_runs, complete_success

In [6]:
# Инициализация окружающей среды
env_name = 'CartPole-v1'
env = gym.make(env_name)

print(env.observation_space.shape[0])
print(env.action_space.n)
print(env._max_episode_steps)

4
2
500


M:\Anaconda3\lib\site-packages\gym\logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [7]:
# Параметры обучения и тестирования
num_runs = 1000
save_condition = 500
num_test_runs = 10

In [8]:
# Инициализация и запуск сети
model_name = 'basic_model/basic.model'
model = create_model(env.observation_space.shape[0], env.action_space.n)
last_run = try_and_learn(env, model, model_name, num_runs, min(save_condition, env._max_episode_steps), True)
if last_run == -1 :
    average_result = 0
    complete_success = 0
else :
    average_result, complete_success = test(env, model_name, num_test_runs, False, True)
print("LR: {}, AR: {}, CS: {}".format(last_run, average_result, complete_success))





run: 0/1000, score: 17/500, eps: 1.0, len(memory): 17
run: 1/1000, score: 16/500, eps: 1.0, len(memory): 33








run: 2/1000, score: 19/500, eps: 0.999, len(memory): 52
run: 3/1000, score: 16/500, eps: 0.991038, len(memory): 68
run: 4/1000, score: 14/500, eps: 0.984123, len(memory): 82
run: 5/1000, score: 18/500, eps: 0.975304, len(memory): 100
run: 6/1000, score: 19/500, eps: 0.96608, len(memory): 119
run: 7/1000, score: 21/500, eps: 0.955987, len(memory): 140
run: 8/1000, score: 35/500, eps: 0.939398, len(memory): 175
run: 9/1000, score: 51/500, eps: 0.915741, len(memory): 226
run: 10/1000, score: 50/500, eps: 0.893125, len(memory): 276
run: 11/1000, score: 27/500, eps: 0.881146, len(memory): 303
run: 12/1000, score: 13/500, eps: 0.875436, len(memory): 316
run: 13/1000, score: 39/500, eps: 0.858526, len(memory): 355
run: 14/1000, score: 35/500, eps: 0.843629, len(memory): 390
run: 15/1000, score: 30/500, eps: 0.831066, len(memory): 420
run: 16/1000, score: 20/500, eps: 0.82279

run: 98/1000, score: 15/500, eps: 0.0236726, len(memory): 500
run: 99/1000, score: 29/500, eps: 0.0233318, len(memory): 500
run: 100/1000, score: 18/500, eps: 0.0231227, len(memory): 500
run: 101/1000, score: 27/500, eps: 0.0228125, len(memory): 500
run: 102/1000, score: 114/500, eps: 0.0215483, len(memory): 500
run: 103/1000, score: 269/500, eps: 0.0188359, len(memory): 500
run: 104/1000, score: 105/500, eps: 0.0178722, len(memory): 500
run: 105/1000, score: 59/500, eps: 0.0173526, len(memory): 500
run: 106/1000, score: 230/500, eps: 0.0154671, len(memory): 500
run: 107/1000, score: 500/500, eps: 0.012045, len(memory): 500
Saving the model as basic_model/basic.model
run: 0/10, score: 500
run: 1/10, score: 500
run: 2/10, score: 500
run: 3/10, score: 500
run: 4/10, score: 500
run: 5/10, score: 500
run: 6/10, score: 500
run: 7/10, score: 500
run: 8/10, score: 500
run: 9/10, score: 500
LR: 107, AR: 500.0, CS: 10


In [9]:
# Раскомментировать и использовать для остановки визуализации, если она была ранее запущена
# env.close()